In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_py import binary_path
import random
import time
import pandas as pd
import random
import requests
from bs4 import BeautifulSoup
from chromedriver_py import binary_path
import pyautogui
import tkinter as tk
from selenium.webdriver.chrome.service import Service
from chromedriver_py import binary_path
import pandas as pd
from sodapy import Socrata
import time, json

In [ ]:
def show_notification(message):
    root = tk.Tk()
    root.geometry("+0+900")  # Positionner la fenêtre en bas à gauche
    root.title("Notification")

    label = tk.Label(root, text=message, font=('Helvetica', 12))
    label.pack()

    root.after(3000, root.destroy)  # La fenêtre disparaît après 3 secondes
    root.mainloop()

# Fonction pour déplacer la souris de manière irrégulière
def move_mouse_irregularly():
    # Position de départ aléatoire
    x_start, y_start = random.randint(100, 500), random.randint(100, 500)
    pyautogui.moveTo(x_start, y_start, duration=random.uniform(0.5, 1.5), tween=pyautogui.easeInOutQuad)
    
    # Faire des mouvements irréguliers (de 3 à 5 mouvements)
    for _ in range(random.randint(3, 5)):
        x_offset = random.randint(-200, 200)
        y_offset = random.randint(-200, 200)
        pyautogui.move(x_offset, y_offset, duration=random.uniform(0.5, 1.5), tween=pyautogui.easeInOutQuad)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from bs4 import BeautifulSoup

try:
    # Lancer le navigateur
    driver = webdriver.Chrome()

    # Charger la page
    driver.get("https://www.zonebourse.com/bourse/actions/amerique-du-sud/colombie-30/")

    # Faire défiler la page lentement (sans mouvements de souris)
    for _ in range(10):  # Effectuer un défilement sur la page
        driver.execute_script("window.scrollBy(0, 300)")  # Défilement de 300px vers le bas
        time.sleep(random.uniform(0.5, 1.5))  # Pause aléatoire entre chaque action

    # Attendre un peu pour simuler un comportement humain avant de scraper
    time.sleep(random.uniform(3, 5))  # Attendre un peu, au cas où la page charge plus lentement

    # Scraper le tableau des actions
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", {"id": "stocks_table"})

    # Extraire les lignes du tableau
    rows = table.find_all("tr", class_="js-ss-row")
    data = []

    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 5:  # Vérifie qu'il y a suffisamment de colonnes
            company_name = cols[1].get_text(strip=True)  # Nom de l'entreprise
            stock_price = cols[2].get_text(strip=True)  # Prix de l'action
            market_cap = cols[3].get_text(strip=True)  # Capitalisation
            one_year_change = cols[4].get_text(strip=True)  # Variation 1 an
            sector = cols[5].get_text(strip=True)  # Secteur

            data.append({
                "Company": company_name,
                "Stock Price": stock_price,
                "Market Cap": market_cap,
                "1 Year Change": one_year_change,
                "Sector": sector
            })

    # Afficher les données extraites
    for entry in data:
        print(entry)

except Exception as e:
    print(f"Une erreur est survenue: {e}")

finally:
    # Fermer le navigateur une fois terminé
    driver.quit()


In [ ]:
df = pd.DataFrame(data)
df.shape

In [ ]:
df

In [ ]:
bourse_col = df.to_json()
bourse_col

In [ ]:
import yfinance as yf

# Liste des symboles boursiers colombiens
tickers_colombie = [
    'CIB', 'PFBCOLOM', 'BVC', 'CNEC', 'EXITO', # Ajoutez les autres symboles ici
]

# Fonction pour vérifier si une entreprise est sous-évaluée avec yfinance
def is_undervalued(ticker):
    try:
        # Récupérer les données de l'entreprise
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Récupérer le P/E Ratio et EPS
        pe_ratio = info.get('trailingPE')
        eps = info.get('trailingEps')
        
        if pe_ratio and eps:
            # Calcul de la valeur théorique de l'action
            theoretical_value = eps * 15  # Un multiple de 15 est souvent utilisé comme référence
            current_price = info.get('previousClose')

            # Vérifier si l'entreprise est sous-évaluée
            if current_price and theoretical_value:
                if current_price < theoretical_value:
                    return True, current_price, theoretical_value, pe_ratio
                else:
                    return False, current_price, theoretical_value, pe_ratio
            else:
                return None, None, None, None
        else:
            return None, None, None, None
    except Exception as e:
        print(f"Erreur avec {ticker}: {e}")
        return None, None, None, None

# Parcourir chaque entreprise et appliquer le calcul
for ticker in tickers_colombie:
    undervalued, current_price, theoretical_value, pe_ratio = is_undervalued(ticker)
    
    if undervalued is not None:
        print(f"\nEntreprise: {ticker}")
        print(f"Prix actuel: {current_price}")
        print(f"Valeur théorique (EPS * 15): {theoretical_value}")
        print(f"P/E Ratio: {pe_ratio}")
        if undervalued:
            print("Cette entreprise est sous-évaluée!")
        else:
            print("Cette entreprise n'est pas sous-évaluée.")
    else:
        print(f"Impossible d'obtenir les données pour {ticker}.")


In [ ]:
with open("govco_data_creds.json", "r") as file:
    content = json.load(file)
    user = content['user']
    token = content['token']
base_url = "www.datos.gov.co"

client = Socrata(base_url, app_token=token)

id_negocios = "jxfg-er2i"
id_droguerias ="y6ye-6gwu"
metadatdroguerias_Mag = client.get_metadata(id_droguerias)
total_records = int(metadatdroguerias_Mag["columns"][0]["cachedContents"]["non_null"])
total_records

In [ ]:
import pandas as pd
from sodapy import Socrata
import time, json

with open("govco_data_creds.json", "r") as file:
    content = json.load(file)
    user = content['user']
    token = content['token']

def get_data_from_gov_col(dataset_id, app_token, limit=10000, max_retries=3):
    import time
    from sodapy import Socrata
    import pandas as pd

    client = Socrata("www.datos.gov.co", app_token=app_token)

    # Récupérer les métadonnées du dataset
    metadata = client.get_metadata(dataset_id)

    # Extraire le nombre total d'enregistrements non nuls pour la première colonne
    total_records = None
    for column in metadata["columns"]:
        if "cachedContents" in column and "non_null" in column["cachedContents"]:
            total_records = int(column["cachedContents"]["non_null"])
            break

    if not total_records:
        print("Impossible de récupérer le nombre total d'enregistrements.")
        return None

    print(f"Nom du dataset : {metadata.get('name')}")
    print(f"Total d'enregistrements dans ce dataset : {total_records}")

    # Initialisation des variables pour le suivi du nombre d'enregistrements collectés
    total_collected = 0
    offset = 0
    all_data = []  # Liste pour stocker les résultats

    # Collecte des données par pages (avec un offset)
    while offset < total_records:
        retries = 0
        while retries < max_retries:
            try:
                results = client.get(dataset_id, limit=limit, offset=offset)

                # Ajouter les résultats collectés à la liste all_data
                all_data.extend(results)

                # Compter les données récupérées dans ce lot
                records_collected = len(results)
                total_collected += records_collected

                # Affichage de l'état d'avancement
                print(f"Enregistrements ajoutés : {records_collected} / Restants : {total_records - total_collected}")

                # Pause de 1 seconde pour ne pas surcharger l'API
                time.sleep(1)

                # Passer à l'offset suivant pour récupérer les données suivantes
                offset += limit
                break  # Sortir du retry loop si la requête réussit

            except Exception as e:
                retries += 1
                print(f"⚠️ Erreur lors de la récupération des données : {e}. Tentative {retries}/{max_retries}")
                time.sleep(5)  # Pause avant de réessayer

                if retries == max_retries:
                    print("❌ Erreurs répétées. Passage à l'offset suivant.")
                    offset += limit  # Éviter le blocage si une page spécifique échoue

    print(f"\nCollecte terminée. Nombre total de données récupérées : {total_collected}")

    # Convertir la liste des données en DataFrame
    df = pd.DataFrame(all_data)

    return df

Nom du dataset : Tiendas Naturistas, Droguerias, Farmacias, Expendios de Suplementos Dietarios Registrados en la Cámara de Comercio de Santa Marta para el Magdalena
Total d'enregistrements dans ce dataset : 1262
Enregistrements ajoutés : 1262 / Restants : 0

Collecte terminée. Nombre total de données récupérées : 1262


In [ ]:
# Exemple d'utilisation
droguerias_mag_id = "y6ye-6gwu"
negocios_mag_id = "jxfg-er2i"  # Remplace par l'ID du dataset que tu veux inspecter

df_droguerias_mag = get_data_from_gov_col(droguerias_mag_id, token)


In [ ]:
negocios_registrados_magdalena_df = get_data_from_gov_col(negocios_mag_id, token)

In [ ]:
CAUSAS_MORTALIDAD_MAGDALENA = "iyp3-g3ks"
df_causas_mortalidad_magdalena = get_data_from_gov_col(CAUSAS_MORTALIDAD_MAGDALENA, token)
df_causas_mortalidad_magdalena.shape

In [ ]:
df_causas_mortalidad_magdalena.head()

In [ ]:
id_calidad_agua_colombia = 'nxt2-39c3'
# metadata_calidad_agua_colombia = client.get_metadata(id_calidad_agua_colombia,token)
df_calidad_agua_colombia = get_data_from_gov_col(id_calidad_agua_colombia, token)
df_calidad_agua_colombia

In [ ]:
id_perdida_investitura_cong_col = 'r6w4-eyn9'
df_perdida_investitura_cong_col = get_data_from_gov_col(id_perdida_investitura_cong_col, app_token=token)
print(df_perdida_investitura_cong_col.shape)
print(df_perdida_investitura_cong_col.columns)
df_perdida_investitura_cong_col.tail()

In [ ]:
id_calidad_aire_magdalena = 'dgnf-6h7v'
df_calidad_aire_magdalena = get_data_from_gov_col(id_calidad_aire_magdalena, app_token=token)
print(df_calidad_aire_magdalena.shape)
print(df_calidad_aire_magdalena.columns)
# Renommer les colonnes pour plus de clarté
df_calidad_aire_magdalena.rename(columns={
    'estacion': 'Station',
    'parametro': 'Pollutant',
    'muestra_no': 'Sample_No',
    'filtro_no': 'Filter_No',
    'fecha': 'Date',
    'pri': 'Start_Pressure_hPa',
    'prf': 'End_Pressure_hPa',
    'pravg': 'Avg_Pressure_hPa',
    'pa': 'Atmospheric_Pressure_hPa',
    'ta_c': 'Temp_Celsius',
    'ta_k': 'Temp_Kelvin',
    'po_pa': 'Flow_Calc',
    'qr_m3_min': 'Air_Flow_m3_min',
    'porcdif': 'Flow_Diff_Percent',
    'li_horometro_min': 'Start_Hourmeter_min',
    'lf_horometro_min': 'End_Hourmeter_min',
    't_min': 'Sampling_Time_min',
    'qstd_m3_min': 'Standard_Flow_m3_min',
    'vstd_m3': 'Standard_Volume_m3',
    'wi_gr': 'Initial_Weight_g',
    'wf_gr': 'Final_Weight_g',
    'wn_gr': 'Net_Weight_g',
    'cpm_g_m3': 'Pollutant_Conc_g_m3'
}, inplace=True)

# Vérifier les nouvelles colonnes
print(df_calidad_aire_magdalena.columns)


df_calidad_aire_magdalena.tail()

In [88]:
id_areas_protegidas_con_ecosistema_de_paramo = 'r2yz-k5z7'
df_areas_protegidas_con_ecosistema_de_paramo = get_data_from_gov_col(id_areas_protegidas_con_ecosistema_de_paramo, app_token=token)
print(df_areas_protegidas_con_ecosistema_de_paramo.shape)
print(df_areas_protegidas_con_ecosistema_de_paramo.columns)
df_areas_protegidas_con_ecosistema_de_paramo.tail()

Nom du dataset : ÁREAS PROTEGIDAS CON ECOSISTEMAS DE PÁRAMO Y GESTIÓN DE LA RAPE REGIÓN CENTRAL
Total d'enregistrements dans ce dataset : 468
Enregistrements ajoutés : 468 / Restants : 0

Collecte terminée. Nombre total de données récupérées : 468
(468, 12)
Index(['c_digo_de_departamento', 'nombre_del_departamento',
       'c_digo_de_municipio', 'nombre_de_municipio', 'nombre_paramo',
       'nombre_rea_protegida',
       'reas_protegidas_del_sistema_nacional_de_areas_protegidas',
       'categor_a_de_reas_protegidas_decreto_2372_2010', 'administrador',
       'fecha_inicio_de_administraci_n', 'rea_hect_reas',
       'ejecuci_n_de_proyectos_de_la_rape_region_central'],
      dtype='object')


,c_digo_de_departamento,nombre_del_departamento,c_digo_de_municipio,nombre_de_municipio,nombre_paramo,nombre_rea_protegida,reas_protegidas_del_sistema_nacional_de_areas_protegidas,categor_a_de_reas_protegidas_decreto_2372_2010,administrador,fecha_inicio_de_administraci_n,rea_hect_reas,ejecuci_n_de_proyectos_de_la_rape_region_central
463,25,Cundinamarca,25279,FOMEQUE,Chingaza,Chingaza,Areas Protegidas Públicas,Parque Nacional Natural,Parques Nacionales Naturales - PNN,1968-06-24T00:00:00.000,0.20,Proyecto Implementación de acciones de conserv...
464,25,Cundinamarca,25279,FOMEQUE,Chingaza,La Reserva,Areas Protegidas Privadas,Reserva Natural de la Sociedad Civil,Parques Nacionales Naturales - PNN,2004-08-11T00:00:00.000,0.20,Proyecto Implementación de acciones de conserv...
465,15,Boyaca,15861,VENTAQUEMADA,Rabanal y rio Bogota,Paramo de Rabanal,Areas Protegidas Públicas,Parques Naturales Regionales,Corporacion Autonoma Regional de Boyaca – CORP...,2009-12-15T00:00:00.000,0.17,Proyecto Implementación de acciones de conserv...
466,15,Boyaca,15861,VENTAQUEMADA,Rabanal y rio Bogota,Paramo Rabanal,Areas Protegidas Públicas,Distritos Regionales de Manejo Integrado,Corporación Autónoma Regional de Chivor – CORP...,2011-02-07T00:00:00.000,0.17,Proyecto Implementación de acciones de conserv...
467,15,Boyaca,15861,VENTAQUEMADA,Rabanal y rio Bogota,Paramo de Rabanal,Areas Protegidas Públicas,Reservas Forestales Protectoras Regionales,Corporación Autónoma Regional de Cundinamarca ...,1992-12-30T00:00:00.000,0.17,Proyecto Implementación de acciones de conserv...


In [89]:
id_especies_amenazadacol = 'wizt-zh64'
df_especies_amenazadacol = get_data_from_gov_col(id_especies_amenazadacol, app_token=token)
print(df_especies_amenazadacol.shape)
print(df_especies_amenazadacol.columns)
df_especies_amenazadacol.tail()

Nom du dataset : Listado oficial de las especies silvestres amenazadas de la diversidad biológica colombiana continental y marino costera - Resolución 0126 de 2024
Total d'enregistrements dans ce dataset : 2103
Enregistrements ajoutés : 2103 / Restants : 0

Collecte terminée. Nombre total de données récupérées : 2103
(2103, 20)
Index(['identificacion', 'identificacion_de_taxonomia', 'nombre_cientifico',
       'clasificacion_superior', 'reino', 'filo', 'clase', 'orden', 'familia',
       'genero', 'epiteto_especifico', 'epiteto_infraespecifico',
       'rango_de_taxon', 'nombre_cientifico_1', 'nombre_vernaculo',
       'codigo_nomenclatura', 'estado_taxonomico', 'taxon_observaciones',
       'lenguaje', 'estado_de_amenaza'],
      dtype='object')


,identificacion,identificacion_de_taxonomia,nombre_cientifico,clasificacion_superior,reino,filo,clase,orden,familia,genero,epiteto_especifico,epiteto_infraespecifico,rango_de_taxon,nombre_cientifico_1,nombre_vernaculo,codigo_nomenclatura,estado_taxonomico,taxon_observaciones,lenguaje,estado_de_amenaza
2098,gbif.org/species/11219159,gbif.org/species/11219159,Trachemys venusta callirostris,Animalia | Chordata | Vertebrata | Reptilia | ...,Animalia,Chordata,Reptilia,Testudines,Emydidae,Trachemys,venusta,callirostris,Subespecie,"(Gray, 1855)",Hicotea,ICZN,Válido,Nombre original validado: Trachemys venusta ca...,es,VU
2099,gbif.org/species/6156842,gbif.org/species/6156842,Kinosternon scorpioides albogulare,Animalia | Chordata | Vertebrata | Reptilia | ...,Animalia,Chordata,Reptilia,Testudines,Kinosternidae,Kinosternon,scorpioides,albogulare,Subespecie,"(Duméril & Bocourt, 1870)",Swanka,ICZN,Válido,Nombre original validado: Kinosternon scorpioi...,es,VU
2100,gbif.org/species/6179762,gbif.org/species/6179762,Crypturellus obsoletus castaneus,Animalia | Chordata | Vertebrata | Aves | Tina...,Animalia,Chordata,Aves,Tinamiformes,Tinamidae,Crypturellus,obsoletus,castaneus,Subespecie,"(P.L.Sclater, 1858)",Tinamú pardo,ICZN,Válido,Nombre original validado: Crypturellus obsolet...,es,CR
2101,gbif.org/species/6175177,gbif.org/species/6175177,Molothrus aeneus armenti,Animalia | Chordata | Vertebrata | Aves | Pass...,Animalia,Chordata,Aves,Passeriformes,Icteridae,Molothrus,aeneus,armenti,Subespecie,"Cabanis, 1851",Chamón del Caribe,ICZN,Válido,Nombre original validado: Molothrus aeneus arm...,es,VU
2102,gbif.org/species/4262310,gbif.org/species/4262310,Odocoileus virginianus tropicalis,Animalia | Chordata | Vertebrata | Mammalia | ...,Animalia,Chordata,Mammalia,Artiodactyla,Cervidae,Odocoileus,virginianus,tropicalis,Subespecie,"Cabrera, 1918",Venado sabanero | Venado cola blanca,ICZN,Válido,Nombre original validado: Odocoileus virginian...,es,CR


In [ ]:
id_asociaciones_recicladores_mag_registrados_cam_Comercio_SM = 'mjqb-b8k4'
df_asociaciones_recicladores_mag_registrados_cam_Comercio_SM = get_data_from_gov_col(id_asociaciones_recicladores_mag_registrados_cam_Comercio_SM, app_token=token)
# Vérification si la variable est None avant d'afficher les informations
if df_asociaciones_recicladores_mag_registrados_cam_Comercio_SM is not None:
    print(df_asociaciones_recicladores_mag_registrados_cam_Comercio_SM.shape)
    print(df_asociaciones_recicladores_mag_registrados_cam_Comercio_SM.columns)
else:
    print("Les données n'ont pas pu être récupérées.")

Nom du dataset : Asociaciones o agrupaciones de Recicladores en la Cámara de Comercio de Santa Marta para el Magdalena
Total d'enregistrements dans ce dataset : 109
Enregistrements ajoutés : 109 / Restants : 0

Collecte terminée. Nombre total de données récupérées : 109
(109, 7)
Index(['matricula', 'est_matricula', 'nit', 'ciiu_1', 'ciiu_2', 'ciiu_3',
       'ciiu_4'],
      dtype='object')


In [100]:
df_test.tail()

,matricula,est_matricula,nit,ciiu_1,ciiu_2,ciiu_3,ciiu_4
104,45627,ACTIVA,ESTABLECIMIENTO,E3812 ** Recoleccion de desechos peligrosos,E3822 ** Tratamiento y disposicion de desechos...,E3900 ** Actividades de saneamiento ambiental ...,N8129 ** Otras actividades de limpieza de edif...
105,63984,ACTIVA,ESTABLECIMIENTO,E3600 ** Captacion tratamiento y distribucion ...,E3700 ** Evacuacion y tratamiento de aguas res...,E3811 ** Recoleccion de desechos no peligrosos,NaN
106,44531,CANCELADA,ESTABLECIMIENTO,E3811 ** Recoleccion de desechos no peligrosos,E3821 ** Tratamiento y disposicion de desechos...,E3822 ** Tratamiento y disposicion de desechos...,N8129 ** Otras actividades de limpieza de edif...
107,211610,INACTIVA,ESTABLECIMIENTO,B0990 ** Actividades de apoyo para otras activ...,G4631 ** Comercio al por mayor de productos al...,E3811 ** Recoleccion de desechos no peligrosos,M7110 ** Actividades de arquitectura e ingenie...
108,215898,INACTIVA,ESTABLECIMIENTO,E3811 ** Recoleccion de desechos no peligrosos,NaN,NaN,NaN
